In [1]:
!pip install -U sentence-transformers
!pip -q install google-generativeai==0.3.0
!pip -q install google-ai-generativelanguage==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e77fe9b7c3010d252ef5f60c8d2fc9e9d2b05574fd2023dcbacf6bfb4f31f56c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_AI_STUDIO = user_secrets.get_secret("GOOGLE_AI_STUDIO")

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

genai.configure(api_key=GOOGLE_AI_STUDIO)

In [3]:
import numpy as np
import requests

from bs4 import BeautifulSoup
from urllib.request import urlopen
from datasets import load_dataset
from sentence_transformers import util
from transformers import pipeline

In [4]:
text = urlopen('https://www.gutenberg.org/cache/epub/277/pg277.txt').read().decode()
docs = list(filter(lambda x: len(x) >100, text.split('\r\n\r\n')))
docs = np.array(docs)
print('There are ',len(docs),' documents/para')

There are  95  documents/para


In [32]:
model = genai.GenerativeModel('gemini-pro')

In [68]:
def get_embedding(doc):
    return genai.embed_content(model='models/embedding-001',content=doc)['embedding']

In [69]:
embeddings = [get_embedding(doc) for doc in docs]

In [70]:
doc_embeddings = np.array(embeddings)

doc_embeddings.shape

(95, 768)

In [71]:
# QUESTION = 'Why was the name TRINITY given to first detonation of a nuclear weapon?'
QUESTION = 'Who was J. Robert Oppenheimer?'

In [72]:
question_embedding = np.array(get_embedding(QUESTION))

In [73]:
hits = util.semantic_search(question_embedding,doc_embeddings,top_k=3)[0]
hits

[{'corpus_id': 17, 'score': 0.7157494501399324},
 {'corpus_id': 14, 'score': 0.7034740733738141},
 {'corpus_id': 41, 'score': 0.6837298329565331}]

In [74]:
print("Question : ",QUESTION)

for i, hit in enumerate(hits):
    print(f'Document {i+1} Cosine_Similarity_Score {hit["score"]:.3f}:\n\n{docs[hit["corpus_id"]]}')
    print("\n")    

Question :  Who was J. Robert Oppenheimer?
Document 1 Cosine_Similarity_Score 0.716:

After witnessing the awesome blast, Oppenheimer quoted a line from a
sacred Hindu text, the Bhagavad-Gita: He said: "I am become death, the
shatterer of worlds."[6] In Los Alamos 230 miles to the north, a group
of scientists' wives who had stayed up all night for the not so secret
test, saw the light and heard the distant sound. One wife, Jane Wilson,
described it this way, "Then it came. The blinding light [no] one had
ever seen. The trees, illuminated, leaping out. The mountains flashing
into life. Later, the long slow rumble. Something had happened, all
right, for good or ill."[7]


Document 2 Cosine_Similarity_Score 0.703:

Workers built three observation points 5.68 miles (10,000 yards), north,
south, and west of Ground Zero. Code named Able, Baker, and Pittsburgh,
these heavily-built wooden bunkers were reinforced with concrete, and
covered with earth. The bunker designated Baker or South 10,000

In [75]:
context = docs[hits[0]['corpus_id']]

PROMPT = f"Given the below context, answer the question.\n\nContext : {context}\nQuery: {QUESTION}\nAnswer:"

In [76]:
print(PROMPT)

Given the below context, answer the question.

Context : After witnessing the awesome blast, Oppenheimer quoted a line from a
sacred Hindu text, the Bhagavad-Gita: He said: "I am become death, the
shatterer of worlds."[6] In Los Alamos 230 miles to the north, a group
of scientists' wives who had stayed up all night for the not so secret
test, saw the light and heard the distant sound. One wife, Jane Wilson,
described it this way, "Then it came. The blinding light [no] one had
ever seen. The trees, illuminated, leaping out. The mountains flashing
into life. Later, the long slow rumble. Something had happened, all
right, for good or ill."[7]
Query: Who was J. Robert Oppenheimer?
Answer:


In [77]:
generation_config = {
  "temperature": 0.7,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,
}


safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_ONLY_HIGH"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_ONLY_HIGH"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_ONLY_HIGH"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_ONLY_HIGH"
  }
]

model = genai.GenerativeModel(model_name="gemini-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


chat = model.start_chat(history=[])

In [78]:
chat.send_message(PROMPT)

Markdown(chat.last.text)

The provided context does not mention J. Robert Oppenheimer, so I cannot answer this question from the provided context.